In [123]:
# imports
from dotenv import load_dotenv
import os
import pandas as pd

In [150]:
# loading environment variables
load_dotenv()
DATABASE_URL = os.getenv('PRODUCTION_DATABASE_URL')

In [125]:
# loading in data
FILE_PATH = os.path.join(os.path.abspath(''), '..', 'raw_data', 'population_data_2010-2019.csv')
teens_raw = pd.read_csv(FILE_PATH, engine='python')

FILE_PATH = os.path.join(os.getcwd(), '..', 'raw_data', 'basic_city_data.txt')
city_geographic_raw = pd.read_csv(FILE_PATH, sep='\t')

In [126]:
teens = teens_raw[['SUMLEV','STATE', 'COUNTY', 'PLACE', 'NAME', 'STNAME', 'POPESTIMATE2010', 'POPESTIMATE2011',
       'POPESTIMATE2012', 'POPESTIMATE2013', 'POPESTIMATE2014',
       'POPESTIMATE2015', 'POPESTIMATE2016', 'POPESTIMATE2017',
       'POPESTIMATE2018', 'POPESTIMATE2019']]
teens['SUMLEV'] = teens['SUMLEV'].apply(lambda x: str(x).zfill(3))
teens['STATE'] = teens['STATE'].apply(lambda x: str(x).zfill(2))
teens['COUNTY'] = teens['COUNTY'].apply(lambda x: str(x).zfill(3))
teens['PLACE'] = teens['PLACE'].apply(lambda x: str(x).zfill(5))
teens.columns = ['level', 'state_id', 'county', 'place', 'city_name', 'state_name', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
teens.head()

,level,state_id,county,place,city_name,state_name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,040,01,000,00000,Alabama,Alabama,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
1,162,01,000,00124,Abbeville city,Alabama,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560
2,162,01,000,00460,Adamsville city,Alabama,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281
3,162,01,000,00484,Addison town,Alabama,751,750,743,742,739,734,731,726,723,718
4,162,01,000,00676,Akron town,Alabama,355,347,347,343,338,339,333,332,331,328


In [127]:
# seperating out the states
states = teens[teens['level'] == '040'].reset_index(drop=True)

In [128]:
# adding in abbreviations
state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

states['state_abbreviation'] = states['state_name'].map(state_abbreviations)

# basic state data
state_basic = states[['state_id', 'state_name', 'state_abbreviation']]
state_basic.head()

,state_id,state_name,state_abbreviation
0,01,Alabama,AL
1,02,Alaska,AK
2,04,Arizona,AZ
3,05,Arkansas,AR
4,06,California,CA


In [129]:
# state population data
state_population = states[['state_id', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']]
state_population = state_population.melt(id_vars=['state_id'])
state_population.columns = ['state_id', 'year', 'population']
state_population.head()

,state_id,year,population
0,01,2010,4785437
1,02,2010,713910
2,04,2010,6407172
3,05,2010,2921964
4,06,2010,37319502


In [131]:
# seperating out city data and basic cleaning
cities = teens[teens['level'] == '162']
cities['type'] = cities['city_name'].apply(lambda x: x.split()[-1])
types_that_are_cities = ['city', 'town', 'village']
cities['type'] = cities['type'].apply(lambda x: 'city' if x.split()[-1] in types_that_are_cities else x)
cities = cities[cities['type'] == 'city']
cities['city_id'] = cities['state_id'] + cities['place']
cities = cities.drop('type', axis=1)
cities['city_name'] = cities['city_name'].apply(lambda x: x.replace(' city', ''))
cities['city_name'] = cities['city_name'].apply(lambda x: x.replace(' town', ''))
cities['city_name'] = cities['city_name'].apply(lambda x: x.replace(' village', ''))
cities.head()

,level,state_id,county,place,city_name,state_name,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,city_id
1,162,01,000,00124,Abbeville,Alabama,2699,2694,2643,2628,2608,2600,2584,2575,2571,2560,0100124
2,162,01,000,00460,Adamsville,Alabama,4500,4493,4471,4449,4420,4390,4356,4327,4308,4281,0100460
3,162,01,000,00484,Addison,Alabama,751,750,743,742,739,734,731,726,723,718,0100484
4,162,01,000,00676,Akron,Alabama,355,347,347,343,338,339,333,332,331,328,0100676
5,162,01,000,00820,Alabaster,Alabama,31209,31375,31684,31980,32182,32772,33017,33275,33413,33487,0100820


In [132]:
# adding in basic city data
city_geographic = city_geographic_raw[['GEOID', 'ALAND_SQMI', 'AWATER_SQMI', 'INTPTLAT', 'INTPTLONG                                                                                                 ']]
city_geographic.columns = ['city_id', 'land_sqmi', 'water_sqmi', 'latitude', 'longitude']
city_geographic['city_id'] = city_geographic['city_id'].apply(lambda x: str(x).zfill(7))
cities = cities.merge(city_geographic, on='city_id', how='left')
city_basic = cities[['city_id', 'city_name', 'state_id', 'land_sqmi', 'water_sqmi', 'latitude', 'longitude']]
city_basic.head()


,city_id,city_name,state_id,land_sqmi,water_sqmi,latitude,longitude
0,0100124,Abbeville,01,15.543,0.042,31.564724,-85.259123
1,0100460,Adamsville,01,26.215,0.209,33.602315,-86.971527
2,0100484,Addison,01,3.766,0.032,34.202681,-87.178004
3,0100676,Akron,01,0.686,0.005,32.879066,-87.740899
4,0100820,Alabaster,01,25.279,0.314,33.244399,-86.847372


In [134]:
# city_population data
city_population = cities[['city_id', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']]
city_population = city_population.melt(id_vars=['city_id'])
city_population.columns = ['city_id', 'year', 'population']
city_population

,city_id,year,population
0,0100124,2010,2699
1,0100460,2010,4500
2,0100484,2010,751
3,0100676,2010,355
4,0100820,2010,31209
...,...,...,...
182525,5681300,2019,467
182526,5683040,2019,3462
182527,5684925,2019,5024
182528,5685015,2019,1753


In [ ]:
'''
tables in db were created using the following:
CREATE TABLE "cities" (
	"city_id" char(7) NOT NULL,
	"city_name" char(50) NOT NULL,
	"state_id" char(2) NOT NULL,
	"latitude" double precision,
	"longitude" double precision,
	"land_sqmi" bigint,
	"water_sqmi" bigint,
	CONSTRAINT "cities_pk" PRIMARY KEY ("city_id")
) WITH (
  OIDS=FALSE
);



CREATE TABLE "state_population" (
	"id" serial NOT NULL,
	"state_id" char(2) NOT NULL,
	"year" bigint NOT NULL,
	"population" bigint NOT NULL,
	CONSTRAINT "state_population_pk" PRIMARY KEY ("id")
) WITH (
  OIDS=FALSE
);



CREATE TABLE "city_population" (
	"id" serial NOT NULL,
	"city_id" char(7) NOT NULL,
	"year" bigint NOT NULL,
	"population" bigint NOT NULL,
	CONSTRAINT "city_population_pk" PRIMARY KEY ("id")
) WITH (
  OIDS=FALSE
);



CREATE TABLE "states" (
	"state_id" char(2) NOT NULL,
	"state_name" char(50) NOT NULL,
	"state_abbreviation" char(2) NOT NULL,
	CONSTRAINT "states_pk" PRIMARY KEY ("state_id")
) WITH (
  OIDS=FALSE
);



ALTER TABLE "cities" ADD CONSTRAINT "cities_fk0" FOREIGN KEY ("state_id") REFERENCES "states"("state_id");

ALTER TABLE "state_population" ADD CONSTRAINT "state_population_fk0" FOREIGN KEY ("state_id") REFERENCES "states"("state_id");

ALTER TABLE "city_population" ADD CONSTRAINT "city_population_fk0" FOREIGN KEY ("city_id") REFERENCES "cities"("city_id");
'''

In [148]:
# adding to db
state_basic.to_sql('states', DATABASE_URL, if_exists='append', index=False)

state_population.to_sql('state_population', DATABASE_URL, if_exists='append', index=False)

city_basic.to_sql('cities', DATABASE_URL, if_exists='append', index=False)

city_population.to_sql('city_population', DATABASE_URL, if_exists='append', index=False)